# Eigencurrent Response Function Testing

In [1]:
import nbimporter
import Current_Filaments

ModuleNotFoundError: No module named 'nbimporter'

In [2]:
class Filament:
    def __init__(self, start, end):
        self.start = start
        self.end = end
    def __repr__(self):
        return "Start: {}, End: {}".format(self.start, self.end)
    
    def BField(self, array_x, array_y, array_z, current):
        """
        array_x, array_y, array_z must be ndarrays of equal shape with at least degree 3
        I is a current value in amperes
        """
        u0 = 1.25663706e-6
        loop_array = np.array([self.start, self.end])
        loop_diff = np.append(np.diff(loop_array, axis=0), [loop_array[0] - loop_array[-1]], axis=0)[:-1]
        loop_shift = np.append(loop_array[1:], [loop_array[0]], axis=0)[:-1]
        loop_array = loop_array[:-1]
        
        point = np.array([array_x, array_y, array_z])
        
        points = np.expand_dims(point, axis=1)
        #### This tile function is fixed to (1, 2, 1, 1, 1) because there are only 2 points in a filament, 
        #### not a closed loop or multi segment
        points = np.tile(points, (1, 2, 1, 1, 1))
        points = points.T
        
        # Vectors from points to endpoints
        AP = points - loop_array
        BP = points - loop_shift
        
        # Field Math
        r1 = np.sqrt((AP ** 2).sum(-1))[..., np.newaxis].T.squeeze().T
        r2 = np.sqrt((BP ** 2).sum(-1))[..., np.newaxis].T.squeeze().T
        Dot1 = np.multiply(AP, loop_diff).sum(-1)
        Dot2 = np.multiply(BP, loop_diff).sum(-1)
        cross = np.cross(loop_diff, AP)
        CrossSqrd = (np.sqrt((cross ** 2).sum(-1))[..., np.newaxis]).squeeze() ** 2
        top = (Dot1 / r1 - Dot2 / r2) * u0 * current
        bottom = (CrossSqrd * 4 * np.pi)
        factor = (top / bottom)
        factor = factor[..., np.newaxis]

        field = cross * factor
        field = np.sum(field, axis=3)
        field = field.T
        return field[0], field[1], field[2]
    
    def get_scatter3d(self, name, mode='lines+markers'):
        line = go.Scatter3d(x=[self.start[0], self.end[0]],
                            y=[self.start[1], self.end[1]],
                            z=[self.start[2], self.end[2]],
                            name=name,
                            mode=mode)
        return line
    
    def get_cones(self, name, x_array, y_array, z_array, current):
        U, V, W = U, V, W = newWire.BField(x_array, y_array, z_array, current)
        cones = go.Cone(x=x_array.flatten(), y=y_array.flatten(), z=z_array.flatten(), 
                        u=U.flatten(), v=V.flatten(), w=W.flatten(), 
                        sizemode='scaled',
                        sizeref=1,
                        anchor='tail',
                        name=name)
        return cones

# initialization function that takes in different parameters instead of a start, stop

def new_filament(start, length, deg_azimuth, deg_phi):
    """
    Create a new filament at P_start with length oriented in the 
    direction specified by azimuth, phi 
    """
    start = np.array(start)
    azi = deg_azimuth * np.pi / 180
    phi = deg_phi * np.pi / 180
    # Translate out of spherical coordinates
    x,y,z = np.sin(azi), np.cos(azi), np.cos(phi)
    dir_hat = np.array([x, y, z]) / np.linalg.norm(np.array([x, y, z]))
    direction = length * dir_hat
    endpoint = start + direction
    return Filament(start, endpoint)

From: https://em.geosci.xyz/content/maxwell3_fdem/circuitmodel_for_eminduction/derive_response_function.html
These EMFs are governed by Faraday’s Law

$$E_j=\frac{−M_{ij}dI_i}{dt}$$

where $E_j$ is the EMF induced in one circuit by a current I_i flowing in another if Mij is their mutual inductance. The EMF induced in the Rx is therefore

In [ ]:
TX_CURRENT = 10
RAMP_TIME = 1500
RAMP_TIME *= 1e-6

dIdt = TX-CURRENT/RAMP_TIME
